# Работа с базами данных

Материалы:
* Макрушин С.В. Лекция 6: Работа с базами данных
* https://sqliteonline.com/
* https://docs.python.org/3/library/sqlite3.html
* https://www.geeksforgeeks.org/sql-join-set-1-inner-left-right-and-full-joins/
* https://www.datacamp.com/community/tutorials/group-by-having-clause-sql

## Задачи для совместного разбора

1. Работая с базой данных `Chinook_Sqlite.sqlite`, найдите и выведите на экран имена и фамилии всех заказчиков из Канады

In [2]:
import sqlite3

In [ ]:
con = sqlite3.connect('Chinook_Sqlite.sqlite')
cur = con.cursor()

cur.execute('''
SELECT firstname, lastname
FROM Customer 
WHERE country=?
''', ('Canada', ))

print(cur.fetchmany(5))

cur.close()
con.close()

In [ ]:
con = sqlite3.connect('Chinook_Sqlite.sqlite')
cur = con.cursor()

cur.execute('''
SELECT *
FROM Customer ''')

print(cur.fetchmany(5))

cur.close()
con.close()

2. Найти и вывести на экран названия всех альбомов группы Accept

In [3]:
con = sqlite3.connect('Chinook_Sqlite.sqlite')
cur = con.cursor()

cur.execute('''
SELECT title
FROM Album 
WHERE ArtistId=?
''', ('''SELECT Artistid
      FROM Artist
      WHERE ArtistId=?
      ''', ('Accept', )))

print(cur.fetchmany(5))

cur.close()
con.close()

ProgrammingError: Incorrect number of bindings supplied. The current statement uses 1, and there are 2 supplied.

3. Создайте базу данных с названием вашей группы. В этой базе данных создайте таблицу Student, содержащую 2 столбца: id и name. Добавьте в таблицу Student информацию о студентах, сидящих с вами по соседству.

In [ ]:
con = sqlite3.connect('pi19-1.db')
cur = con.cursor()

cur.execute('''
CREATE TABLE Student(id INTEGER, name VARCHAR(20))
''')

cur.execute('''
INSERT INTO Student(id, name) VALUES (1, 'Anna')
''')
con.commit()


cur.close()
con.close()

## Лабораторная работа 6

1. Создайте файл базы данных c названием `recipes.db`. Создайте объект-курсор. 

In [4]:
import  sqlite3
con = sqlite3.connect("recipes.db")
cur = con.cursor()
# con.close() 

2. Напишите и выполните SQL-запрос для создания таблицы рецептов `Recipe`. Таблица должна содержать следующие поля:
`id`, `name`, `minutes`, `submitted`, `description`, `n_ingredients`. Определитесь с типами и составом ключевых полей.

In [2]:
cur.execute('DROP TABLE Recipe')
cur.execute('''CREATE TABLE Recipe 
               (id INTEGER PRIMARY KEY NOT NULL, 
               name VARCHAR(200), 
               minutes INTEGER, 
               submitted VARCHAR(10), 
               description VARCHAR(1000), 
               n_ingredients INTEGER)''')

3. Напишите и выполните SQL-запрос для создания таблицы отзывов `Review`. Таблица должна содержать следующие поля:
`id`, `user_id`, `recipe_id`, `date`, `rating`, `review`. Определитесь с типами полей, набором ключевых полей. При помощи внешнего ключа соедините две таблицы.

In [4]:
cur.execute('DROP TABLE Review')
cur.execute('''CREATE TABLE Review 
               (id INTEGER PRIMARY KEY NOT NULL, 
               user_id INTEGER, 
               recipe_id INTEGER, 
               date VARCHAR(10), 
               rating REAL, 
               review VARCHAR(1000))''')

4. Загрузите данные из файлов `reviews_sample.csv` (__ЛР2__) и `recipes_sample_with_tags_ingredients.csv` (__ЛР5__) в созданные таблицы

In [27]:
con.execute("""drop table if exists recipe""")
con.execute("""drop table if exists review;""")

In [28]:
import pandas as pd
reviews = pd.read_csv("../02_pandas/data/reviews_sample.csv", index_col=0)

In [29]:
reviews.to_sql("Review", con)

In [30]:
recipes = pd.read_csv("../05_data_files/recipes_sample_with_tags_ingredients.csv", index_col=0)
recipes.to_sql("recipe", con)

5. Найдите все рецепты, для выполнения которых нужно ровно 10 ингредиентов. Выведите на экран первые 5 из найденных рецептов.

In [10]:
seq = cur.execute('''SELECT * FROM Recipe 
                        WHERE n_ingredients = 10
                        LIMIT 0, 5''')

In [11]:
seq.fetchall()

[(6,
  503475,
  'blepandekager   danish   apple pancakes',
  50,
  128473,
  '2013-07-08',
  10,
  "this recipe has been posted here for play in zwt9 - scandinavia.  this recipe was found at website: mindspring.com - christian's danish recipes.",
  10,
  10,
  'danish; 60-minutes-or-less; time-to-make; course; cuisine; preparation; pancakes-and-waffles; breakfast; scandinavian; european',
  'milk*eggs*butter*flour*salt*cream*sugar*apples*cinnamon*lemon, juice of'),
 (9,
  125195,
  'kelly s creamy cheddar pea salad',
  20,
  113941,
  '2005-06-09',
  5,
  "i'm not a big fan of peas, but like them in things. this salad is one of my favorites. happy to share!",
  10,
  12,
  '30-minutes-or-less; time-to-make; course; cuisine; preparation; north-american; salads; side-dishes; american; easy; midwestern; 3-steps-or-less',
  'onion*celery*cheddar cheese*mayonnaise*sugar*frozen peas*hard-boiled eggs*sweet pickle relish*seasoning salt*mustard'),
 (12,
  120297,
  'middle eastern   twice bake

6. Найдите название рецепта, для выполнения которого требуется больше всего времени.

In [12]:
seq = cur.execute('''SELECT name FROM Recipe 
                        WHERE minutes IN 
                            (SELECT MAX(minutes) FROM Recipe) ''')

In [13]:
seq.fetchall()

[('strawberry liqueur',)]

7. Запросите у пользователя id рецепта и верните информацию об этом рецепте. Если рецепт отсуствует, выведите соответствующее сообщение.

In [14]:
def user_input():
    id_ = input("Введите id: ")
    try:
        id_ = int(id_)
        return id_
    except ValueError:
        print('Введите число типа int')
        return user_input()

In [16]:
# id_ = input("Введите id: ")
id_ = user_input()
seq = cur.execute(f'SELECT * FROM Recipe WHERE id = {id_}',)  
rez = seq.fetchone()
if not rez:
    print(f'Рецепт с id = {id_} отсутствует')
else:
    print(rez)

Введите id: 120297
(12, 120297, 'middle eastern   twice baked potatoes', 110, 197023, '2005-05-02', 14, 'a delicious yet low-fat twice-baked potato side dish with a blend of classic middle eastern flavours: chickpeas, cumin and coriander.  serve with roasts or with your favourite middle eastern main course dish.  adapted from "practical cookery: low fat".  these potatoes could also be served on their own as a light vegetarian meal. in view of the comments by reviewers about the potatoes being a bit dry, i have added a tablespoon of tahini. i have also added some garlic. i\'m not sure how the recipe escaped having garlic in it when i first posted it!', 10, 24, 'time-to-make; course; main-ingredient; cuisine; preparation; occasion; healthy; side-dishes; potatoes; vegetables; asian; middle-eastern; dinner-party; low-fat; vegetarian; lebanese; dietary; low-sodium; low-cholesterol; low-saturated-fat; low-calorie; healthy-2; low-in-something; 4-hours-or-less', 'coriander*garlic cloves*olive 

8. Найдите кол-во отзывов с рейтингом 5.

In [31]:
seq = cur.execute('''SELECT COUNT(*) FROM Review
                        WHERE rating = 5''')

In [32]:
seq.fetchall()

[(91361,)]

9. Найдите кол-во уникальных рецептов, не имеющих отзывов с рейтингом, меньше 4. 

In [33]:
seq = cur.execute('''SELECT COUNT(DISTINCT name) FROM Recipe
                        WHERE id IN
                            (SELECT recipe_id FROM Review
                                WHERE rating >= 4)''')

In [34]:
seq.fetchall()

[(16457,)]

10. Найдите кол-во рецептов, опубликованных в 2010 году и имеющих длину не менее 15 минут.

In [36]:
seq = cur.execute('''SELECT COUNT(*) FROM Recipe
                        WHERE strftime('%Y', submitted) = '2010' AND
                        minutes >= 15''')

In [37]:
seq.fetchall()

[(819,)]

11. Выберите id рецепта, название рецепта, id пользователя, оставившего отзыв, дату отзыва и рейтинг для тех рецептов, которые имеют не менее 3 ингредиентов. Отсортируйте результат по id рецепта.

In [38]:
seq = cur.execute('''SELECT rc.id, rc.name, rv.user_id, rv.date, rv.rating
                        FROM Recipe rc
                        INNER JOIN Review rv
                        ON rc.id = rv.recipe_id
                        WHERE rc.n_ingredients >= 3
                        ''')

In [39]:
seq.fetchall()

[(84797, 'love is in the air  beef fondue   sauces', 11176, '2006-01-11', 0),
 (84797, 'love is in the air  beef fondue   sauces', 110905, '2004-12-26', 5),
 (84797, 'love is in the air  beef fondue   sauces', 123871, '2007-02-26', 5),
 (84797, 'love is in the air  beef fondue   sauces', 128047, '2009-01-01', 4),
 (84797, 'love is in the air  beef fondue   sauces', 198228, '2005-04-06', 5),
 (84797, 'love is in the air  beef fondue   sauces', 257993, '2007-02-16', 5),
 (84797, 'love is in the air  beef fondue   sauces', 509740, '2008-05-30', 3),
 (84797, 'love is in the air  beef fondue   sauces', 560694, '2008-03-25', 5),
 (107229, 'open sesame  noodles', 133842, '2007-06-12', 5),
 (95926, 'say what   banana sandwich', 17803, '2005-04-10', 5),
 (95926, 'say what   banana sandwich', 26032, '2004-07-21', 5),
 (95926, 'say what   banana sandwich', 29782, '2005-06-23', 5),
 (95926, 'say what   banana sandwich', 33722, '2004-07-22', 5),
 (95926, 'say what   banana sandwich', 54271, '2005-0